In [47]:
import numpy as np
import pandas as pd

def load_e2_chunk(csv_file, start_row, num_rows=20, channels=19):
    """
    Reads a CSV file and fetches `num_rows` starting from `start_row`,
    reshapes into WxHxC for CNN input. Non-numeric values are replaced with 0.
    """
    df = pd.read_csv(csv_file, header=None)
    
    # Convert all to numeric, replace errors with 0
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    data_chunk = df.iloc[start_row:start_row + num_rows, :channels].values  # shape: (num_rows, channels)
    
    # WxH grid
    W = 5
    H = 4
    
    # Check if padding is needed
    total_elements = W * H
    flat_data = data_chunk.flatten()  # shape: num_rows*channels
    if flat_data.size < total_elements * channels:
        padded = np.zeros(total_elements * channels)
        padded[:flat_data.size] = flat_data
    else:
        padded = flat_data[:total_elements * channels]  # truncate if larger
    
    # Reshape into WxHxC
    grid = padded.reshape(W, H, channels)
    
    return grid.astype(np.float32)  # final shape: 5x4xC


def load_srs_chunk(csv_path, input_rows=70, pred_offset=5, W=20, H=14):
    """
    Sliding window SRS generator.
    
    Each CSV row contains a string representation of a list: "[[...],[...],[...],[...]]"
    """
    df = pd.read_csv(csv_path, header=None)
    
    # Parse string lists into actual arrays
    srs_data = []
    for val in df[1]:  # assuming column 1 contains the lists
        arr = np.array(ast.literal_eval(val), dtype=np.float32)  # shape (4, 1536)
        srs_data.append(arr)
    
    srs_data = np.stack(srs_data, axis=0)  # shape (num_rows, 4, 1536)
    
    num_rows = srs_data.shape[0]
    
    for t in range(num_rows - input_rows - pred_offset + 1):
        X_seq = srs_data[t:t+input_rows]  # shape (input_rows, 4, 1536)
        # reshape to WxHxC
        X_seq_reshaped = np.stack([x.reshape(W, H, 1536) for x in X_seq], axis=0)
        
        y = srs_data[t + input_rows + pred_offset - 1]  # shape (4, 1536)
        yield X_seq_reshaped.astype(np.float32), y.astype(np.float32)

In [48]:
from tensorflow.keras.models import Model

def train_model(model, e2_csv, srs_csv, epochs=1, batch_size=1):
    """
    Train the model using sliding window chunks from E2 and SRS CSVs.
    Collects loss history for plotting.
    """
    # Read CSVs
    e2_rows = pd.read_csv(e2_csv).shape[0]
    srs_rows = pd.read_csv(srs_csv, header=None).shape[0]
    
    # Define sliding window lengths
    e2_len = 20
    srs_input_rows = 70
    pred_offset = 1  # predict next row

    # Number of training steps
    steps = min(e2_rows - e2_len, srs_rows - srs_input_rows - pred_offset + 1)

    loss_history = []

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # Reset the SRS generator
        srs_gen = load_srs_chunk(srs_csv, input_rows=srs_input_rows, pred_offset=pred_offset)

        for step in range(steps):
            # --- Load E2 chunk ---
            X_e2 = load_e2_chunk(e2_csv, step, num_rows=e2_len)
            X_e2_batch = np.expand_dims(X_e2, axis=0)  # add batch dim

            # --- Load SRS chunk ---
            try:
                X_srs, y = next(srs_gen)
            except StopIteration:
                break  # generator exhausted
            X_srs_batch = np.expand_dims(X_srs, axis=0)
            y_batch = np.expand_dims(y, axis=0)

            # --- Adjust SRS shape to match model input ---
            # Example: model expects (20,20,1536)
            # Here we crop/pad W/H if needed
            target_W, target_H, target_C = model.input[1].shape[1:]  # srs_input_shape
            X_srs_resized = X_srs_batch
            if X_srs_batch.shape[1] != target_W or X_srs_batch.shape[2] != target_H:
                # Simple crop or pad to match
                X_srs_resized = X_srs_batch[:, :target_W, :target_H, :]

            # --- Train step ---
            loss = model.train_on_batch([X_e2_batch, X_srs_resized], y_batch)
            loss_history.append(loss)

            if (step+1) % 50 == 0 or step == steps-1:
                print(f"Step {step+1}/{steps}, Loss: {loss:.6f}")

    return loss_history


In [49]:
from tensorflow.keras import layers, models

def create_model(e2_input_shape=(5, 4, 19), srs_input_shape=(20, 20, 1536), lstm_units=128, dropout_rate=0.3):
    
    radio_input = layers.Input(shape=e2_input_shape, name='radio_input')
    
    r1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(radio_input)  # use padded r1
    r1 = layers.BatchNormalization()(r1)
    r1 = layers.Conv2D(32, (2, 2), activation='relu', padding='same')(r1)
    r1 = layers.BatchNormalization()(r1)
    r1 = layers.Dropout(dropout_rate)(r1)
    r1 = layers.ZeroPadding2D(padding=((0,0),(0,1)))(r1)  # pad width by 1

    srs_input = layers.Input(shape=srs_input_shape, name='srs_input')
    s1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(srs_input)
    s1 = layers.Conv2D(1024, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.BatchNormalization()(s1)
    s1 = layers.MaxPooling2D((2, 2))(s1)
    s1 = layers.Conv2D(32, (2, 2), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(256, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(128, (2, 2), activation='relu', padding='same')(s1)
    s1 = layers.BatchNormalization()(s1)
    s1 = layers.MaxPooling2D((2, 2))(s1)
    s1 = layers.Conv2D(128, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Dropout(dropout_rate)(s1)

    # Concatenate and LSTM
    x = layers.Concatenate(axis=-1)([r1, s1])
    x = layers.Reshape((5*5, 32+64))(x)
    x = layers.LSTM(lstm_units, return_sequences=True)(x)
    x = layers.Flatten()(x)         
    x = layers.Dense(4*1536)(x)      
    output = layers.Reshape((4,1536))(x)

    model = models.Model(inputs=[radio_input, srs_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model


In [50]:
model = create_model()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 srs_input (InputLayer)         [(None, 20, 20, 153  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_65 (Conv2D)             (None, 20, 20, 64)   884800      ['srs_input[0][0]']              
                                                                                                  
 conv2d_66 (Conv2D)             (None, 20, 20, 1024  66560       ['conv2d_65[0][0]']              
                                )                                                                 
                                                                                            

In [51]:
train_model(model, 
            "P:/SP Challenge/DataSet/Preprocessed Dataset/E2_1.csv", 
            "P:/SP Challenge/DataSet/Preprocessed Dataset/pp_srs_1.csv", 
            epochs=3, 
            batch_size=1)

Epoch 1/3


KeyError: 1